In [9]:
import numpy as np
from keras.layers import Dense, Input, Lambda
from keras.models import Model
from keras.optimizers import adadelta
from keras.regularizers import l1
import pydot
import matplotlib.pyplot as plt 
import keras.backend as K

In [10]:
def shallow_variational_autoencoder(input_size,embed_size,batch_size):
    
    # encoder 
    
    x_in = Input(batch_shape=(batch_size,input_size))
    X=Dense(512,activation='relu')(x_in)
    Z_mu=Dense(embed_size,activation='relu')(X)
    Z_sigma=Dense(embed_size,activation='relu')(X)
    
    def sampling(args):
        Z_mu , Z_sigma = args
        epsilon=K.random_normal(shape=(batch_size,embed_size), mean=0.0, stddev=1.0)
        return Z_mu+K.exp(Z_sigma)*epsilon
    
    
    Z = Lambda(sampling, output_shape=(embed_size,))([Z_mu , Z_sigma])
    
    # decoder
    
    decode_1=Dense(512,activation='relu')
    decode_2=Dense(input_size,activation='sigmoid')
    
    h1=decode_1(Z)
    X_mean = decode_2(h1)
    
    
    VAE=Model(inputs = x_in , outputs = X_mean)
    
    #encoder model
    
    encoder = Model (inputs = x_in,outputs= Z_mu )
    
    
    # decoder model
    
    
    decoder_input = Input(shape=(embed_size,))
    _h_decoded = decode_1(decoder_input)
    _x_decoded_mean = decode_2(_h_decoded)
    generator = Model(inputs = decoder_input, outputs= _x_decoded_mean )
   
    
    return VAE,encoder, generator


VAE,encoder, generator = shallow_variational_autoencoder(input_size= 784 ,embed_size = 64, batch_size= 128)

In [11]:
VAE.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (128, 784)           0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (128, 512)           401920      input_4[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (128, 64)            32832       dense_8[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (128, 64)            32832       dense_8[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

In [15]:
from keras.utils import plot_model
plot_model(VAE, to_file='VAE_model.png')

In [ ]:
def vae_loss(X_in, X_fake):
    log_loss=K.binary 
    